## 第五章  pandas入门

###### pandas含有是数据清洗和分析工作变得更加简单的数据结构和操作工具。
    pandas常和其它工具一起使用，如数值计算工具Numpy和Scipy，分析库statsmodels和scikit-learn，数据可视化库matplotlib。pandas是基于numpy数组构建的，特别是基于数组的函数和不使用for循环的处理数据。
    pandas专门处理表格和混杂数据，numpy更适合处理统一的数值数组数据

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas import Series
from pandas import DataFrame

### 5,1 pandas 的数据结构介绍
    两个主要的数据结构：Series和DataFrame

###### 5.1.1  Series
    Series是一种类似于一维数组的对象，它是由一组数据（各种Numpy数据类型）以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单的Series:

In [11]:
obj = pd.Series([4, 7, -5, 3])

In [5]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

    表现形式：索引在左边，值在右边。Series的values和index属性获取数组表现形式的索引对象。

In [6]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [7]:
obj.index   # like range(4)

RangeIndex(start=0, stop=4, step=1)

    为Series创建一个可以对各个数据点进行标记的索引：

In [8]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

In [9]:
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [10]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

    通过索引方式选取Series中的单个或一组值

In [11]:
obj2['a']

-5

In [13]:
obj2['d'] = 6

In [14]:
obj2[['c', 'a', 'd']]  # 传入索引列表

c    3
a   -5
d    6
dtype: int64

    使用numpy函数或类似numpy的运算。都会保留索引值的链接

In [15]:
obj2[obj2 > 0] # 布尔型数组进行过滤

d    6
b    7
c    3
dtype: int64

In [16]:
obj2 * 2  # 标量乘法

d    12
b    14
a   -10
c     6
dtype: int64

In [19]:
np.exp(obj2)  # 以自然常数e为底的指数函数，返回e的幂次方

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

    还可以将Series看成一个定长的有序字典，因为他是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

In [20]:
'b' in obj2

True

In [21]:
'e' in obj2

False

    如果数据被存放在一个python字典中，也可以通过这个字典来创建Series

In [3]:
sdata= {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

In [4]:
obj3 = pd.Series(sdata)

In [24]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

    传入排好序的字典的键以改变顺序

In [5]:
states = ['California', 'Ohio', 'Oregon', 'Texas']

In [6]:
obj4 = pd.Series(sdata, index=states)

In [33]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

    索引相匹配的3个值会被找出来并放到相应的位置上，找不到相匹配值的键所对应的结果就是NaN（非数字），在pandas中，用于表示缺失或NA

    用缺失或NA表示缺失数据。pandas的isnull和notnull函数可以用于检测缺失值

In [34]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [35]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [36]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [37]:
obj4.notnull()

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

    Series的一个重要功能是，它会根据运算的索引标签自动对齐数据

In [38]:
obj3  # 四个索引

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [39]:
obj4  # 四个索引，与obj3相同的索引有三个

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [40]:
obj3 + obj4
# 相同索引的值对应相加了，不存在相同索引的相当于加了一个没定义的值，即为NaN

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

    Series对象本身机器索引都有一个name属性，该属性根pandas的其他关键功能关系很密切

In [7]:
obj4.name = 'population'

In [9]:
obj4.index.name = 'state'

In [10]:
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

    Series的索引可以通过赋值的方式就地修改

In [12]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [13]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']

In [14]:
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

###### 5.1.2 DataFrame
    DataFrame是一个表格型数据结构，它含有一组有序的列，每列可以是不同的值类型（数值，字符串，布尔值等）。DataFrame既有行索引也有列索引，可以看做是一个由Series组成的字典（共用同一个索引）。DataFrame中的数据是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）。

    建DataFrame：直接传入一个由等长列表或numpy数组组成的字典。

In [16]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada','Nevada' ,'Nevada'],
    'year': [2000, 2001, 2002, 2001, 2002, 2003],
    'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]
}
frame = pd.DataFrame(data)

In [17]:
# DataFrame会自动加上索引（根Series一样），全部列都会被有序排列
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [18]:
frame.head()
# head()不带参数时默认显示前五行

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


    如果指定了列序号，DataFrame的列就会按照指定顺序进行排列：

In [19]:
pd.DataFrame(data, columns=['year','state','pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


    如果传入的列在数据中找不到，就会在结果中产生缺失值：

In [17]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                     index=['one', 'two', 'three', 'four', 'five', 'six'])

In [21]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [22]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

    通过类似字典标记的方式或属性的方式，可以将DataFrame的列获取为一个Series:
    注意：返回的Series拥有原DataFrame相同的索引，且其name属性也已经被相应的设置好了。

In [23]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [24]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

    frame2[columns]适用于任何列的名，但是frame2.columns只有在列名是一个合理的python变量名时才适用

    行也可以通过位置或名称的方式进行获取，比如用loc属性

In [25]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

    列可以通过赋值的方式进行修改。

In [28]:
# frame2['debt'] = 16.5
frame2.debt = 16.5
# debt是合理的python变量，以上两种方式都可以

In [29]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [34]:
frame2['debt'] = np.arange(6.)

In [35]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


    为不存在的列赋值会创建出一个新列。关键字del用于删除列。

In [42]:
frame2['eastern'] = frame2.state == 'Ohio'
# frame2.eastern = frame2.state == 'Ohio'
# 不能用frame2.eastern创建新的列 ？？？ 为什么

In [43]:
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,0.0,True
two,2001,Ohio,1.7,1.0,True
three,2002,Ohio,3.6,2.0,True
four,2001,Nevada,2.4,3.0,False
five,2002,Nevada,2.9,4.0,False
six,2003,Nevada,3.2,5.0,False


In [44]:
del frame2['eastern']

In [20]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [46]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

    注意：通过索引方式返回的列只是相应数据的视图而已，并不是副本。因此，对返回的Series所做的任何就地修改都会反映到源DataFrame上。通过Series的copy方法可以指定赋值列。

    另一种常见的数据形式是嵌套字典：

In [8]:
pop = {
    'Nevada': {
        2001: 2.4,
        2002: 2.9
    },
    'Ohio':{
        2000: 1.5,
        2001: 1.7,
        2002: 3.6
    }
}

    如果嵌套字典传给DataFrame，pandas就会被解释为：外层字典的键作为列，内层字典的键则作为行索引。

In [9]:
frame3 = pd.DataFrame(pop)

In [6]:
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


    使用类似numpy数组的方法，对DataFrame进行转置（交换行和列）

In [7]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


    内层字典的键会被合并、排序以形成最终的索引。如果明确指定了索引，则不会这样

In [8]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


    由Series组成的字典差不多也是一样的用法

In [9]:
pdata = {
    'Ohio': frame3['Ohio'][:1],
    'Nevada': frame3['Nevada'][:2]
}

In [10]:
pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,NaN,2.4


    可以输入给DataFrame构造器的数据：
        二维ndarray；
        由数组、列表或元祖组成的字典；
        numpy的结构化/记录数组；
        由Series组成的字典；
        由字典组成的字典；
        字典或Series的列表；
        由列表或元祖组成的列表；另一个DataFrame；
        numpy的MaskedArray;

    如果设置了DataFrame的index和columns的name属性，则这些信息也会被显示出来：

In [11]:
frame3.index.name = 'year';
frame3.columns.name = 'state'

In [13]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


    根Series一样，values属性也会以二维ndarray的形式返回DataFrame中的数据：

In [14]:
frame3.values

array([[nan, 1.5],
       [2.4, 1.7],
       [2.9, 3.6]])

    如果DataFrame各列的数据类型不同，则值数组的dtype就会选用能兼容所有列的数据类型。

In [18]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, nan],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, nan],
       [2002, 'Nevada', 2.9, nan],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

###### 5.1.3 索引对象
    pandas的索引对象负责管理轴标签和其它元数据（比如轴名称等）。构建Series或DataFrame时，所用到的任何数组或其他序列标签都会被转换成一个index

In [21]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])

In [22]:
index = obj.index

In [23]:
index

Index(['a', 'b', 'c'], dtype='object')

In [25]:
index[1:]

Index(['b', 'c'], dtype='object')

    index对象是不可变的，因此用户不能对其进行修改的，不可变可以使index对象在多个数据结构之间安全共享。
    虽然不需要经常使用Index的功能，但是因为一些操作会生成包含被所索引化的数据，还是需要理解工作原理。

In [4]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [5]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)

In [6]:
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [7]:
obj2.index is labels

True

    除了类似于数组，Index的功能也类似于一个固定大小的集合。

In [12]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [15]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [16]:
'Ohio' in frame3.columns

True

In [18]:
2003 in frame3.columns

False

    与python的集合不同，pandas的Index可以包含重复的标签。
    选择重复的标签，会显示所有的结果。

In [19]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])

In [20]:
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

    每个索引都有一些方法和属性，它们可用于设置逻辑并回答有关该索引所包含的数据的常见问题

|方法|说明|
|---|---|
|append|链接另一个Index对象，产生一个新的Iindex|
|difference|计算差集，并得到一个Index|
|intersection|计算交集|
|union|计算并集|
|isin|计算一个指示各值是否都包含在参数集合中的布尔型变量|
|delete|删除索引i处的元素，并得到新的Index|
|drop|删除传入的值，并得到新的Index|
|insert|将元素插入到索引i处，并得到新的Index|
|is_monotonic|当各元素均大于等于前一个元素时，返回True|
|is_unique|当Index没有重复值时，返回True|
|unique|计算Index中唯一值的数组|

### 5.2  基本功能
    介绍操作Series和DataFrame中的数据的基本手段。

##### 5.2.1  重新索引
    pandas对象的一个重要方法是reindex，其作用是创建一个新对象，它的数据符合新的索引。

In [21]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])

In [22]:
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

    用该Series的reindex将会根据索引进行排序。如果某一个索引值当前不存在，就引入缺失值。

In [23]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])

In [24]:
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

    对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理，method选项即可达到此目的，例如，使用ffill可以实现前向值填充

In [25]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])

In [26]:
obj3

0      blue
2    purple
4    yellow
dtype: object

In [27]:
obj3.reindex(range(6), method='ffill')
# 前向值填充

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

    借助DataFrame，reindex可以修改（行）索引和列。只传递一个序列时，会重新索引结果的行

In [30]:
frame = pd.DataFrame(np.arange(9).reshape((3,3)), index=['a','c','d'], 
                     columns=['Ohio', 'Texas', 'California'])

In [31]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [32]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])

In [33]:
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


    列可以用关键字重新索引：

In [34]:
states = ['Texas', 'Utah', 'California']

In [35]:
frame.reindex(columns=states)
# 没有了原来的Ohio列，新增了Utah列，是原来没有的，所以以NaN填充。

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


            reindex函数的各种参数说明

| 参数 | 说明 |
| :---: | :---: |
| index | 用作索引的新序列。既可以是index实例，也可以其他序列型的python数据结构。Index会完全被使用，就像没有任何复制一样 |
| method | 插值（填充）方式 |
| fill_value | 在重新索引的过程中，需要引入缺失值时使用的替代值 |
| limit | 前向或后向填充时的最大填充量 |
| tolerance | 向前后向后填充时，填充不准确匹配项的最大间距（绝对值距离） |
| level | 在MultiIndex的指定级别上匹配简单索引，否则选取其子集 |
| copy | 默认为True，无论如何都复制；如果为False，则新旧相等就不复制 |

##### 5.2.2 丢弃指定轴上的项

    丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。
    由于需要执行一些数据整理或集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的新对象。

In [3]:
obj = pd.Series(np.arange(5.), index=['a','b','c','d','e'])

In [4]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [5]:
new_obj = obj.drop('c')

In [8]:
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [9]:
obj.drop(['d','c'])

a    0.0
b    1.0
e    4.0
dtype: float64

    对于DataFrame，可以删除任意轴上的索引值。

In [10]:
data = pd.DataFrame(np.arange(16).reshape((4,4)), 
                    index=['Ohio', 'Colorado', 'Utah', 'NewYork'],
                   columns=['one', 'two', 'three', 'four'])

In [11]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
NewYork,12,13,14,15


    用标签序列调用drop会从行标签（axis 0）删除值：

In [12]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
NewYork,12,13,14,15


    通过传递axis=1,或axis='columns'可以删除列的值：
    PS：也就是说，drop默认是按行来删除值，行---axis=0；传递了axis设置其他值可以删除列。当删除多行或者多列时，以列表的形式来传递参数。

In [13]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
NewYork,12,14,15


In [15]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
NewYork,12,14


    许多函数，如drop，会修改Series或DataFrame的大小或形状，可以就地修改对象，不会返回新的对象。
    PS：使用inplace的时候要小心，它会销毁所有被删除的数据。

In [16]:
obj.drop('c', inplace=True)

In [17]:
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

##### 5.2.3 索引、选取和过滤
    Series索引（obj[...]）的工作方式类似于numpy数组的索引，只不过Series的索引值不只是整数。

In [18]:
obj = pd.Series(np.arange(4.), index=['a','b','c','d'])

In [19]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [20]:
obj['b']

1.0

In [21]:
obj[1]

1.0

In [22]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [23]:
obj[['b', 'a', 'd']]
# 按照传入的索引列表元素顺序来显示

b    1.0
a    0.0
d    3.0
dtype: float64

In [24]:
obj[[1,3]]

b    1.0
d    3.0
dtype: float64

In [25]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

    利用标签的切片运算与普通的python切片运算不同，其末端是包含的：

In [26]:
obj['b':'c']
#  python的切片运算是 左取右不取 
#  标签的切片运算是 左取右也取

b    1.0
c    2.0
dtype: float64

    利用切片可以对Series的相应部分进行设置：

In [27]:
obj['b':'c'] = 5

In [28]:
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

    用一个值或序列就DataFrame进行索引其实就是获取一个或多个列：

In [4]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                   index=['Ohio', 'Colorado', 'Utah', 'NewYork'],
                   columns=['one', 'two', 'three', 'four'])

In [30]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
NewYork,12,13,14,15


In [31]:
data['two']
# 获取了 two  这一列

Ohio         1
Colorado     5
Utah         9
NewYork     13
Name: two, dtype: int32

In [32]:
data[['three', 'one']]
# 多列以列表形式传入

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
NewYork,14,12


    这种索引有几个特殊的情况。首先通过切片或布尔型数组选取数据：

In [33]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [34]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
NewYork,12,13,14,15


    选取行的语法data[:2]十分方便。向[]传递单一的元素或列表，就可以选择列。

    另一种用法是通过布尔型DataFrame(比如下面这个由标量比较运算得出的)进行索引：
    这使得DataFrame与numpy二维数组的语法很像。

In [35]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
NewYork,False,False,False,False


In [36]:
data[data < 5] = 0

In [37]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
NewYork,12,13,14,15


##### 5.2.4 用loc和iloc进行选取
    对于DataFrame的行的标签索引，我引入了特殊的标签运算符loc和iloc。它们可以让你用类似numpy的标记，使用轴标签（loc）或整数索引（iloc），从DataFrame选择行和列的子集。

In [38]:
# loc----通过轴标签选取---参数为轴标签
data.loc['Colorado', ['two', 'three']]
# 选择了一行和多列：行--Colorado；列--two，three

two      5
three    6
Name: Colorado, dtype: int32

In [39]:
# 用iloc和整数进行选取---参数为整数
data.iloc[2, [3, 0, 1]]
# 第 2 行，第 3 0 1 列（实际上是第 3 行 ；第 4 1 2 列）
# 总共选取了一行数据，以一维数组显示展示。

four    11
one      8
two      9
Name: Utah, dtype: int32

In [40]:
data.iloc[2]
# 选取了第3行

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [42]:
data.iloc[[1, 2], [3,0,1]]
# 选取了  2 3 行，4 1 2 列
# 总共选取了2行，以二维数组的形式展示

,four,one,two
Colorado,7,0,5
Utah,11,8,9


    这两个索引函数也适用于一个标签或多个标签的切片：

In [43]:
data.loc[:'Utah', 'two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [46]:
data.iloc[:, :3][data.three > 5]
# 选取所有行， 选取前3列；在此基础上再筛选 three 列大于 5 的行

,one,two,three
Colorado,0,5,6
Utah,8,9,10
NewYork,12,13,14


    在pandas中，有多个方法可以选取和重新组合数据。

            对于DataFrame，有如下的操作：
|类型|说明|
|---|---|
|df[val]|从DataFrame选取单列或一组列，在特殊情况下比较便利，布尔型数组（过滤）、切片（行切片）、或布尔型DataFrame（根据条件设置值）|
|df.loc[val]|通过标签，选取DataFrame的单个行或一组行（列表形式的参数）|
|df.loc[:, val]|通过标签，选取单列或列子集（列表中第一个元素表示选取了所有行）|
|df.iloc[where]|通过整数位置，从DataFrame选取单个行或行子集|
|df.iloc[:, where]|通过整数位置，从DataFrame选取单个列或列子集（列表中第一个元素表示选取了所有行）|
|df.iloc[where_i, where_j]|通过整数位置，同时获取行和列|
|df.at[label_i, label_j]|通过行和列标签，选取单一的标量|
|df.iat[i, j]|通过行和列的位置（整数），选取单一的标量|
|reindex|通过标签选取行或列|
|get_value, set_value|通过行和列标签选取单一值|

In [5]:
data.at['Utah', 'two']

9

In [6]:
data.iat[2,1]

9

In [12]:
data.get_value('Utah', 'two')

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.


9

In [13]:
data.set_value('Utah', 'two', 11)

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,11,10,11
NewYork,12,13,14,15


##### 5.2.5 整数索引
    pandas对象的整数索引，与python内置的列表和元祖的索引语法不同。

In [14]:
ser =pd.Series(np.arange(3.))
ser  #  正确输出
# ser[-1]  # KeyError, 这是python内置列表的用法；我们的数据是 0 1 2 的索引，有歧义

0    0.0
1    1.0
2    2.0
dtype: float64

    对于非整数索引，不会产生歧义：

In [16]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])

In [18]:
ser2

a    0.0
b    1.0
c    2.0
dtype: float64

In [17]:
ser2[-1]
# 索引是非整数的，这里使用的索引不会产生歧义，选取出了数据

2.0

    为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确，建议使用loc（标签）或iloc（整数）

In [22]:
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [19]:
ser[:1]
# 普通的切片，左取右不取

0    0.0
dtype: float64

In [20]:
ser.loc[:1]
# 标签的切片运算：左取右也取；
# 列表里只有一个元素，选取的是行

0    0.0
1    1.0
dtype: float64

In [23]:
ser.iloc[:1]
# 列表只有一个元素，选取的是行，这里是第一行
# 这是整数索引，不是标签索引，只有标签索引才是左右都取

0    0.0
dtype: float64

##### 5.2.6 算术运算和数据对齐

In [3]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])

In [4]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])

In [5]:
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [6]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [7]:
s1 + s2
# s1 的 a c d e
# s2 的 a c e f g

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

    自动的数据对齐操作在不重叠的索引处引入了NA值。缺失值会在算术运算过程中传播

In [8]:
# 对于DataFrame，对齐操作会同时发生在行和列上：
df1 = pd.DataFrame(np.arange(9.).reshape((3,3)),columns=list('bcd'),
                  index=['Ohio', 'Texas', 'Colorado'])

In [10]:
df2 = pd.DataFrame(np.arange(12.).reshape((4,3)),columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [11]:
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [12]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [13]:
# 相加之后会产生一个新的DataFrame
# 其索引和列为原来那两个DataFrame的并集：
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


    如果DataFrame对象相加，没有共用的列或行标签，结果都会是空：

In [14]:
df1 = pd.DataFrame({'A': [1,2]})

In [15]:
df2 = pd.DataFrame({'B': [3,4]})

In [16]:
df1

,A
0,1
1,2


In [17]:
df2

,B
0,3
1,4


In [18]:
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


##### 5.2.7 在算术方法中填充值
    在对不同索引的对象进行算术填充时，你可能希望当一个对象中某个标签在另一个对象中找不到时填充一个特殊值（比如0）：

In [20]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),columns=list('abcd'))

In [21]:
df2 = pd.DataFrame(np.arange(20.).reshape((4,5)), columns=list('abcde'))

In [22]:
df2.loc[1, 'b'] = np.nan

In [23]:
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [25]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


    将两个DataFrame相加时，没有重叠的位置就会产生NA值：

In [26]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


    使用df1的add方法，传入df2以及一个fill_value参数：

In [28]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


    Series和DataFrame每一个算术方法都有一个副本，以字母r开头，它会翻转参数，下面的两个语句是等价的

In [29]:
1 / df1

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


In [30]:
df1.rdiv(1)

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


|方法|说明|
|---|---|
|add，radd|用于加法（+）的方法|
|sub，rsub|用于减法（-）的方法|
|div，rdiv|用于除法（/）的方法|
|floordiv，rfloordiv|用于底除（//）的方法|
|mul，rmul|用于乘法（*）的方法|
|pow，rpow|用于指数（**）的方法|

In [32]:
df1.rfloordiv(2)
# 2除数据，结果取整，2/0  2/1  2/2  2/3  2/4  2/5

,a,b,c,d
0,NaN,2.0,1.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


##### 5.2.8 DataFrame和Series之间的运算
    根不同维度的numpy数组一样，DataFrame和Series之间算术运算也是有明确规定的。

    例子：计算一个二维数组与其某行之间的差：

In [33]:
arr = np.arange(12.).reshape((3,4))

In [34]:
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [35]:
arr[0]

array([0., 1., 2., 3.])

In [38]:
arr - arr[0]
# 一行一行的减，每一行相对应的位置减

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

    当我们从arr减去arr[0]，每一行都会执行这个操作，这就叫广播（broadcasting）。DataFrame和Series之间的运算差不多也是如此。

In [5]:
frame = pd.DataFrame(np.arange(12.).reshape((4,3)), columns=list('bde'),
                    index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [6]:
series = frame.iloc[0]

In [41]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [42]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

    默认情况下，DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着行一直向下广播。

In [43]:
frame - series
# series 的索引： b d e
# 0 1 2 - 0 1 2  = 0 0 0
# 3 4 5 - 0 1 2  = 3 3 3
# 6 7 8 - 0 1 2  = 6 6 6
# 9 10 11 - 0 1 2 = 9 9 9 

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


    如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集：

In [8]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])

In [9]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


    如果希望匹配行且在列上广播，则必须使用算术运算方法。

In [10]:
series3 = frame['d']

In [11]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [12]:
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [13]:
frame.sub(series3, axis='index')
# sub函数，减法，设置了axis='index'，按照列来进行算术运算

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


##### 5.2.9 函数应用和映射
    numpy的ufuncs（元素级数组方法）也可以用于操作pandas对象：

In [14]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'), 
                    index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [15]:
frame

,b,d,e
Utah,-0.541663,0.007783,0.147465
Ohio,0.307274,1.117785,0.432284
Texas,0.143908,0.082073,-0.562951
Oregon,-1.247795,0.293926,-1.803917


In [16]:
np.abs(frame)
# 求绝对值

,b,d,e
Utah,0.541663,0.007783,0.147465
Ohio,0.307274,1.117785,0.432284
Texas,0.143908,0.082073,0.562951
Oregon,1.247795,0.293926,1.803917


    另一个常见的操作，将函数应用到各列或行所形成的一维数据。DataFrame的apply方法即可实现这一功能。

In [ ]:
f = lambda x : x.max() - x.min()
# f函数计算了一个Series的最大值和最小值的差，在frame的每列都执行了一次。
# 结果是一个Series，使用frame的列作为索引

In [18]:
frame.apply(f)
# 默认在列上面执行

b    1.555069
d    1.110001
e    2.236201
dtype: float64

In [19]:
# 如果传递axis='columns' 到 apply 中，这个函数会在每行中执行。
frame.apply(f, axis='columns')

Utah      0.689128
Ohio      0.810511
Texas     0.706859
Oregon    2.097843
dtype: float64

    许多最为常见的数组统计功能都被实现成DataFrame的方法（sum，mean），因此不需要使用apply方法。

    传递到apply的函数不是必须返回一个标量，还可以返回由多个值组成的Series：

In [20]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

In [21]:
frame.apply(f)

,b,d,e
min,-1.247795,0.007783,-1.803917
max,0.307274,1.117785,0.432284


    元素级的python函数也是可以用的。比如，得到frame中各个浮点值的格式化字符串，使用applymap即可

In [22]:
format = lambda x : '%.2f' % x

In [23]:
frame.applymap(format)

,b,d,e
Utah,-0.54,0.01,0.15
Ohio,0.31,1.12,0.43
Texas,0.14,0.08,-0.56
Oregon,-1.25,0.29,-1.80


    之所以叫做applymap，是因为Series有一个用于应用元素级函数的map方法：

In [24]:
frame['e'].map(format)

Utah       0.15
Ohio       0.43
Texas     -0.56
Oregon    -1.80
Name: e, dtype: object

##### 5.2.10 排序和排名
    根据条件对数据集排序（sorting）也是一种重要的内置函数。要对行或列索引进行排序（按字典排序），可以使用sort_index方法，它将返回一个已排序的新对象：

In [25]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])

In [26]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

    对于DataFrame，则可以根据任意一个轴上的索引进行排序：

In [27]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)), index=['three', 'one'],
                    columns=['d', 'a', 'b', 'c'])

In [28]:
frame.sort_index()
# 按照行进行排列，降序

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [29]:
frame.sort_index(axis=1)
# 设置了axis=1， 按照列来排序，升序

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [30]:
# 默认升序，也可以改变，按照降序排列
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


    若要按值对Series进行排序，可以使用期sort_values方法：

In [31]:
obj = pd.Series([4,7,-3,2])

In [33]:
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

In [34]:
# 在排序中，任何缺失值默认都会被放到Series的末尾
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])

In [35]:
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

    当排序一个DataFrame时，你可能希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给sort_values的by选项即可达到该目的：

In [36]:
frame = pd.DataFrame({
    'b': [4,7,-3,2],
    'a': [0,1,0,1]
})

In [37]:
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [38]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


In [39]:
# 要根据多个列进行排序，传入名称的列表即可：
frame.sort_values(by=['a', 'b'])
# a在前面，先按照a进行排列，相同的再按照b进行排列

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


    排名会从1开始一直到数组中有效数据的数量。
    Series和DataFrame的rank方法。默认情况下，rank是通过‘为各组分配一个平均排名’的方法破坏平级关系的：

In [41]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])

In [42]:
obj.rank()
#  数据：-5   0   2   4   4    7    7
#  下标：1    5   4   3   6    0    2
#  顺序：1    2   3   4   5    6    7 
#  rank：1    2   3  4.5 4.5  6.5  6.5

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [43]:
# 也可以根据值在源数据出现的顺序给出排名
obj.rank(method='first')
# 这里，条目0和2没有使用平均排名6.5，它们被设成了6和7，因为数据中标签0位于标签2的前面

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [45]:
# 降序排序
obj.rank(ascending=False, method='max')
# ascending 参数

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

    DataFrame可以在行或列上计算排名：

In [46]:
frame = pd.DataFrame({
    'b': [4.3, 7, -3, 2],
    'a': [0, 1, 0, 1],
    'c': [-2, 5, 8, -2.5]
})

In [47]:
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [48]:
frame.rank(axis='columns')

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


##### 5.2.11 带有重复标签的轴索引

In [49]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])

In [50]:
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

    索引的is_unique属性可以检验它的值是否是唯一的：

In [51]:
obj.index.is_unique

False

    对于带有重复值的索引，数据选取的行为将会有些不同。如果某个索引对应多个值，则会返回一个Series；而对于对应单个值的，则返回一个标量值：

In [52]:
obj['a']

a    0
a    1
dtype: int64

In [53]:
obj['c']

4

    索引的输出类型会根据标签是否重复发生变化，使得代码变复杂

In [56]:
# 对DataFrame的行进行索引时也是如此：
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])

In [57]:
df

,0,1,2
a,0.255631,0.021229,0.538400
a,-1.664147,-0.914953,0.442728
b,0.297116,-0.385296,1.087193
b,0.029138,0.854536,-0.480711


In [58]:
df.loc['b']

,0,1,2
b,0.297116,-0.385296,1.087193
b,0.029138,0.854536,-0.480711


### 5.3 汇总和计算描述统计

In [3]:
df = pd.DataFrame([[1.4, np.nan], [7.1, 4.5], [np.nan, np.nan], [0.75, -1.3]],
                 index=['a', 'b', 'c', 'd'],
                 columns=['one', 'two'])

In [60]:
df

,one,two
a,1.40,NaN
b,7.10,4.5
c,NaN,NaN
d,0.75,-1.3


In [61]:
# 调用DataFrame的sum方法将会返回一个含有列的和的Series：
df.sum()

one    9.25
two    3.20
dtype: float64

In [62]:
# 传入axis='columns'或axis=1将会按行进行求和运算：
df.sum(axis='columns')

a     1.40
b    11.60
c     0.00
d    -0.55
dtype: float64

    NA 值会自动被排除，除非整个切片（这里指的是行或列）都是NA。通过skipna选项可以禁用该功能：

In [5]:
df.mean(axis='columns', skipna=False)

a      NaN
b    5.800
c      NaN
d   -0.275
dtype: float64

    有些方法（如idxmin和idxmax）返回的是间接统计（比如达到最小值或最大值的索引）

In [7]:
df.idxmax()

one    b
two    b
dtype: object

In [8]:
#  累计型 方法：
df.cumsum()
# 该行的值 = 该行的值 + 上一行的值

,one,two
a,1.40,NaN
b,8.50,4.5
c,NaN,NaN
d,9.25,3.2


    还有一种方法，它既不是约简型也不是累计型。describe就是一个例子，用于一次性产生多个汇总统计：

In [9]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,1.600000
std,3.493685,4.101219
min,0.750000,-1.300000
25%,1.075000,0.150000
50%,1.400000,1.600000
75%,4.250000,3.050000
max,7.100000,4.500000


    对于非数值型数据，describe会产生另外一种汇总统计：

In [10]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)

In [11]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

##### 5.3.1 唯一值、值计数以及成员资格

In [14]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

    函数unique（），可以得到Series中的唯一值数组：

In [15]:
uniques = obj.unique()

In [16]:
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

    返回的唯一值时未排序的，如果需要的话，可以对结果再次进行排序(uniques.sort()).相似的，value_counts用于计算一个Series中各值出现的频率

In [17]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

    为了便于查看，结果Series是按照值频率降序排序的。value_counts还是一个顶级pandas方法，可用于任何数组或序列：

In [18]:
pd.value_counts(obj.values, sort=False)
#  不排序

d    1
c    3
b    2
a    3
dtype: int64

    isin用于判断矢量化集合的成员资格，可用于过滤Series中或DataFrame列中数据的子集

In [19]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [20]:
mask = obj.isin(['b', 'c'])

In [21]:
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [22]:
obj[mask]
# 根据布尔型数组来选取obj中的数据

0    c
5    b
6    b
7    c
8    c
dtype: object

    与isin类似的是Index.get_indexer方法，可以返回一个索引数组，从可能包含重复值的数组到另一个不同值的数组：

In [23]:
to_match= pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])

In [24]:
unique_vals = pd.Series(['c', 'b', 'a'])

In [25]:
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2], dtype=int64)

In [26]:
data = pd.DataFrame({
    'Qu1': [1,3,4,3,4],
    'Qu2': [2,3,1,2,3],
    'Qu3': [1,5,2,4,4]
})

In [27]:
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [28]:
result = data.apply(pd.value_counts).fillna(0)
# value_counts计算各值出现的频率

In [29]:
result
# 值 1 2 3 4 5 
# 在每一列中出现的频率

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
